1. Load weights of DeepFace
2. Add classification layer (2)
3. For each syn in syn_list calculate the stats


In [1]:
import keras
import numpy as np
from os import path
from os import listdir
from os.path import isfile, join
from PIL import Image 
from keras.models import Model
import tensorflow as tf
#tf.compat.v1.disable_v2_behavior()
import csv
import pandas as pd
from tqdm import tqdm
import keras.initializers

from sklearn.model_selection import LeaveOneOut
import cv2
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.preprocessing import Normalizer

Using TensorFlow backend.


In [2]:
IMAGE_SIZE = (152, 152) # set by the model 
CHANNELS = 3 # RGB image
NUM_CLASSES = 8631 # classification layer will be removed 
LEARN_RATE = 0.01
MOMENTUM = 0.9

DOWNLOAD_PATH = 'https://github.com/swghosh/DeepFace/releases/download/weights-vggface2-2d-aligned/VGGFace2_DeepFace_weights_val-0.9034.h5.zip'
MD5_HASH = '0b21fb70cd6901c96c19ac14c9ea8b89'

In [3]:
wt_init = keras.initializers.RandomNormal(mean=0, stddev=0.01)
bias_init = keras.initializers.Constant(value=0.5)

def conv2d_layer(**args):
    return keras.layers.Conv2D(**args, 
        kernel_initializer=wt_init, 
        bias_initializer=bias_init,
        activation=keras.activations.relu)
def lc2d_layer(**args):
    return keras.layers.LocallyConnected2D(**args, 
        kernel_initializer=wt_init, 
        bias_initializer=bias_init,
        activation=keras.activations.relu)
def dense_layer(**args):
    return keras.layers.Dense(**args, 
        kernel_initializer=wt_init, 
        bias_initializer=bias_init)

In [4]:
def create_classifying_deepface(image_size=IMAGE_SIZE, channels=CHANNELS, num_classes=NUM_CLASSES, learn_rate=LEARN_RATE, momentum=MOMENTUM):
    deepface = keras.models.Sequential([
        keras.layers.InputLayer(input_shape=(*image_size, channels), name='I0'),
        conv2d_layer(filters=32, kernel_size=11, name='C1'),
        keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same',  name='M2'),
        conv2d_layer(filters=16, kernel_size=9, name='C3'),
        lc2d_layer(filters=16, kernel_size=9, name='L4'),
        lc2d_layer(filters=16, kernel_size=7, strides=2, name='L5'),
        lc2d_layer(filters=16, kernel_size=5, name='L6'),
        keras.layers.Flatten(name='F0'),
        dense_layer(units=4096, activation=keras.activations.relu, name='F7'),
        keras.layers.Dropout(rate=0.5, name='D0'),
        dense_layer(units=num_classes, activation=keras.activations.softmax, name='F8')
    ], name='DeepFace')
    # deepface.summary()

    sgd_opt = keras.optimizers.SGD(lr=learn_rate, momentum=momentum)
    cce_loss = keras.losses.categorical_crossentropy

    #deepface.compile(optimizer=sgd_opt, loss=cce_loss, metrics=['accuracy'])
    
    return deepface

In [5]:
def get_weights():
    filename = 'deepface.zip'
    downloaded_file_path = keras.utils.get_file(filename, DOWNLOAD_PATH, 
        md5_hash=MD5_HASH, extract=True)
    downloaded_h5_file = path.join(path.dirname(downloaded_file_path), 
        path.basename(DOWNLOAD_PATH).rstrip('.zip'))
    return downloaded_h5_file


def create_deepface():
    model = create_classifying_deepface()
    weights = get_weights()
    model.load_weights(weights)
    
    num_classes = 2
    x = model.layers[-2].output
    x = dense_layer(units=num_classes, activation=keras.activations.softmax, name='preds')(x)
    #x = Dense(2, activation='softmax', name='predictions')(x)
    model2 = Model(model.input, x)
    
    sgd_opt = keras.optimizers.SGD(lr=LEARN_RATE, momentum=MOMENTUM)
    cce_loss = keras.losses.categorical_crossentropy
    #model2.compile(optimizer=sgd_opt, loss=cce_loss, metrics=['accuracy'])
 

    model2.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=keras.optimizers.Adam(lr=0.001),
        metrics=["sparse_categorical_accuracy"])
    
    #model2.summary()
    
    return model2

In [6]:
def load_data(syn, GENERAL_DIR):
    
    data, labels = [], [] 
    
    syn_dir = GENERAL_DIR + "\\{}\{}-patients".format(syn, syn)
    ID_dir = GENERAL_DIR + "\\{}\{}-selected-ID-controls".format(syn, syn)

    # get list of filenames
    files_syn = [f for f in listdir(syn_dir) if (isfile(join(syn_dir, f)))and ".jpg" in f]
    files_ID = [f for f in listdir(ID_dir) if (isfile(join(ID_dir, f))) and ".jpg" in f]
    
    print("Syn_list: {}, ID_list: {}".format(len(files_syn), len(files_ID)))

    for filename in files_syn:
        im = Image.open(join(syn_dir, filename))
        im = im.resize(IMAGE_SIZE)
        data.append(np.array(im))
        labels.append(1)

    for filename in files_ID:
        im = Image.open(join(ID_dir, filename))
        im = im.resize(IMAGE_SIZE)
        data.append(np.array(im))
        labels.append(0)    
    
    return np.array(data), np.array(labels)

In [10]:
def make_gradcam_heatmap(
    img_array, model, last_conv_layer_name, classifier_layer_names
):
    
    img_array = tf.convert_to_tensor(img_array, dtype=np.float32)
    # First, we create a model that maps the input image to the activations
    # of the last conv layer
    last_conv_layer = model.get_layer(last_conv_layer_name)
    last_conv_layer_model = keras.Model(model.inputs, last_conv_layer.output)

    # Second, we create a model that maps the activations of the last conv
    # layer to the final class predictions
    classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)
    classifier_model = keras.Model(classifier_input, x)

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        # Compute activations of the last conv layer and make the tape watch it
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        # Compute class predictions
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    # This is the gradient of the top predicted class with regard to
    # the output feature map of the last conv layer
    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    # The channel-wise mean of the resulting feature map
    # is our heatmap of class activation
    heatmap = np.mean(last_conv_layer_output, axis=-1)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap

In [11]:
model  = create_deepface()
model.summary()

img = Image.open(r"H:\Genetica Projecten\Facial Recognition\Studenten en Onderzoekers\Fien\test\YY1_1.jpg")
img = np.array(img.resize(IMAGE_SIZE))
img = np.expand_dims(img, axis=0)
print(img.shape)

preds = model.predict(img)
print("predictions: {}".format(preds))

last_conv_layer_name = "C3"
classifier_layer_names = ["L4", "L5", "L6", "F0", "F7", "D0", "preds"]

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
I0 (InputLayer)              (None, 152, 152, 3)       0         
_________________________________________________________________
C1 (Conv2D)                  (None, 142, 142, 32)      11648     
_________________________________________________________________
M2 (MaxPooling2D)            (None, 71, 71, 32)        0         
_________________________________________________________________
C3 (Conv2D)                  (None, 63, 63, 16)        41488     
_________________________________________________________________
L4 (LocallyConnected2D)      (None, 55, 55, 16)        62774800  
_________________________________________________________________
L5 (LocallyConnected2D)      (None, 25, 25, 16)        7850000   
_________________________________________________________________
L6 (LocallyConnected2D)      (None, 21, 21, 16)        2829

In [12]:
heatmap = make_gradcam_heatmap(img, model, last_conv_layer_name, classifier_layer_names)

plt.matshow(heatmap)
plt.show()

AssertionError: 

In [14]:
from platform import python_version
print(python_version())

3.7.7


In [13]:
print(tf.__version__) ## 2.2.0
print(keras.__version__)

2.2.0
2.3.1


In [ ]:
plt.figure()
plt.plot(heatmap)
plt.show()